In [ ]:
import requests
import json
import random
import pandas as pd # Import pandas for easier data handling
from pathlib import Path

# --- CONFIGURATION ---
key = open("openweatherkey.txt", "r")
WEATHER_API_KEY = key.read()
key.close()

OUTPUT_FILE = Path("dataset/personalized_clothing_dataset_female.json") 

# --- Load Content from JSON ---
try:
    with open(OUTPUT_FILE, 'r') as f:
        # Load the JSON data into a Python list of dictionaries
        wardrobe_data_list = json.load(f)

    # Convert the list into a Pandas DataFrame for easy filtering and access
    df_wardrobe = pd.DataFrame(wardrobe_data_list)
    
    print(f"Successfully loaded {len(df_wardrobe)} records from the dataset.")
    print("\n--- Sample Data (DataFrame Head) ---")
    print(df_wardrobe.head())

except FileNotFoundError:
    print(f"Error: The file {OUTPUT_FILE} was not found. Please ensure it is in the correct directory.")
except json.JSONDecodeError:
    print(f"Error: Could not decode JSON from {OUTPUT_FILE}. The file may be empty or corrupted.")

Using Weather API Key: fdb90e23cb96ec81fc855e57fd189ca5
Successfully loaded 20 records from the dataset.

--- Sample Data (DataFrame Head) ---
                                          image_link  category  \
0  simulated_wardrobes/Female_Wardrobe/Tops_04_Ka...   t-shirt   
1  simulated_wardrobes/Female_Wardrobe/Outers_03_...      coat   
2  simulated_wardrobes/Female_Wardrobe/Bottoms_08...     jeans   
3  simulated_wardrobes/Female_Wardrobe/Bottoms_07...     skirt   
4  simulated_wardrobes/Female_Wardrobe/Bottoms_04...  trousers   

      outer_inner                                              shape  \
0           inner  {'sleeve': 'short-sleeve', 'neckline': 'crew-n...   
1           outer  {'sleeve': 'long-sleeve', 'neckline': 'collar'...   
2           inner  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   
3  not-applicable  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   
4           inner  {'sleeve': 'none', 'neckline': 'none', 'fit': ...   

                      mater

In [18]:
## --- WEATHER FETCHING ---
def get_weather(city):
    """Fetches weather data from OpenWeatherMap and extracts key values."""
    if not WEATHER_API_KEY:
        print("ERROR: Please set your OpenWeatherMap API key.")
        return None
        
    url = (
        f"https://api.openweathermap.org/data/2.5/weather"
        f"?q={city}&appid={WEATHER_API_KEY}&units=metric"
    )

    try:
        response = requests.get(url)
        response.raise_for_status() # Raise exception for HTTP errors (4xx or 5xx)
        data = response.json()
        
        temp = data["main"]["temp"]
        wind = data["wind"]["speed"]
        rain_1h = data.get("rain", {}).get("1h", 0) # mm in last 1 hour
        
        weather_desc = data["weather"][0]["description"]
        
        print(f"Weather in {city}: {temp:.1f}°C, {weather_desc}")
        
        return {
            "city": city,
            "temp": temp,
            "wind": wind,
            "rain": rain_1h,
            "desc": weather_desc
        }
        
    except requests.exceptions.HTTPError as e:
        print(f"ERROR fetching weather for {city}: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

## --- ENVIRONMENTAL SCORING ---
def compute_required_scores(weather_data):
    """
    Translates temperature, wind, and rain into required garment scores (1-5 for warmth/layering, 1-3 for impermeability).
    """
    temp = weather_data["temp"]
    rain = weather_data["rain"]
    wind = weather_data["wind"]

    # Warmth Requirement (Higher score = warmer clothes needed)
    # Scale: Cold (1) to Hot (5) is reversed to required warmth (5) to (1).
    if temp < 5:      warmth_req = 5 # Very cold
    elif temp < 15:   warmth_req = 4 # Cool
    elif temp < 25:   warmth_req = 3 # Mild
    elif temp < 32:   warmth_req = 2 # Warm
    else:             warmth_req = 1 # Hot

    # Adjust for wind chill (simplified)
    if wind > 8: warmth_req += 1
    warmth_req = min(5, warmth_req)
    
    # Impermeability Requirement (1=none, 3=high)
    if rain >= 2.5: # Heavy rain
        impermeability_req = 3
    elif rain > 0.5: # Light rain/drizzle
        impermeability_req = 2
    else:
        impermeability_req = 1
        
    # Layering Base (The system encourages layering if temperature is volatile or cold)
    # We'll use 4 for default layering to promote flexibility unless it's hot.
    layering_req = 4 if warmth_req >= 3 else 3

    return {
        "warmth": warmth_req,
        "impermeability": impermeability_req,
        "layering": layering_req
    }

In [ ]:
GARMENT_TYPES = {
    "Outer": ["outer"],
    "Top": ["inner"],
    "Bottom": ["inner", "not-applicable"] # Bottoms (jeans, shorts) and dresses are marked as 'not-applicable' in layering, so we must include both.
}
# Map garment category names to required component
CATEGORY_MAP = {
    "jacket": "Outer", "coat": "Outer", "hoodie": "Outer",
    "t-shirt": "Top", "button-up shirt": "Top", "sweater": "Top",
    "jeans": "Bottom", "trousers": "Bottom", "shorts": "Bottom", "skirt": "Bottom",
    "dress": "Dress"
}

def rank_garments(garments_df, required_scores, component_type):
    """
    Ranks garments based on how well their attributes meet or exceed the required scores.
    """
    df = garments_df.copy()
    
    # Find all categories that map to the current component type (e.g., 'Top' -> ['t-shirt', 'sweater', 'polo'])
    valid_categories = [cat for cat, comp in CATEGORY_MAP.items() if comp == component_type]
    
    if not valid_categories:
        # Should not happen if CATEGORY_MAP is comprehensive
        return []

    df_filtered = df[df['category'].isin(valid_categories)].copy()
    
     # Filter by the correct 'outer_inner' type
    if component_type == "Dress":
        df_filtered = df[df['category'] == 'dress'].copy()
    else:
        # Filter for Tops/Outers/Bottoms
        df_filtered = df_filtered[df_filtered['outer_inner'].isin(GARMENT_TYPES[component_type])].copy()


    if df_filtered.empty:
        return []
        
    # Calculate fitness score:
    # Warmth: Minimize the difference between item warmth and required warmth.
    df_filtered['warmth_fit'] = 10 - abs(df_filtered['warmth_score'] - required_scores['warmth'])
    
    # Impermeability: Must meet or exceed the requirement. Penalty if too low.
    impermeability_penalty = 5 # Max penalty for zero impermeability when 3 is needed
    df_filtered['impermeability_fit'] = df_filtered['impermeability_score'].apply(
        lambda x: 10 if x >= required_scores['impermeability'] else 10 - impermeability_penalty
    )

    # Layering: Favor items with high layering score, especially if required layering is high.
    df_filtered['layering_fit'] = df_filtered['layering_score']
    
    # Total Score: Sum the fits, with more weight on Warmth and Impermeability.
    df_filtered['total_score'] = (
        df_filtered['warmth_fit'] * 0.5 + 
        df_filtered['impermeability_fit'] * 0.3 + 
        df_filtered['layering_fit'] * 0.2
    )

    # Apply a hard rule: if component is a Dress, no Top/Bottom is needed.
    if component_type == "Dress" and not df_filtered.empty:
        # For a dress, only return the highest-scoring dress.
        best_dress = df_filtered.sort_values(by='total_score', ascending=False).iloc[0]
        return {"Dress": best_dress.to_dict()}
        
    # Return the top 3 best-fitting items for variety
    return df_filtered.sort_values(by='total_score', ascending=False).head(3).to_dict('records')


def recommend_outfit(wardrobe_df, city):
    """Finds the best combination of Outer, Top, and Bottom for the weather."""
    
    weather_data = get_weather(city)
    if not weather_data:
        return "Failed to get weather data."
        
    required_scores = compute_required_scores(weather_data)
    print("\nRequired Scores:", required_scores)
    
    # --- Check for a Dress first ---
    dress_ranking = rank_garments(wardrobe_df, required_scores, "Dress")
    if dress_ranking and len(dress_ranking) > 0 and required_scores['warmth'] <= 3:
        # If a dress is available and it's not too cold (Warmth Req <= 3), recommend it
        return {
            "Outfit Type": "Dress",
            "Weather": weather_data,
            "Required Scores": required_scores,
            "Dress": dress_ranking["Dress"] 
        }

    # --- Assemble Layered Outfit (Outer + Top + Bottom) ---
    
    # Get ranked items for each component
    top_ranks = rank_garments(wardrobe_df, required_scores, "Top")
    bottom_ranks = rank_garments(wardrobe_df, required_scores, "Bottom")
    outer_ranks = rank_garments(wardrobe_df, required_scores, "Outer")
    print("\nTop Ranks:", top_ranks)
    print("\nBottom Ranks:", bottom_ranks)
    
    if not top_ranks or not bottom_ranks:
        return "ERROR: Missing essential items (Top or Bottom) in the wardrobe."

    # Select the best combination (simply pick the highest scoring item from each component)
    
    # Choose Top
    best_top = top_ranks[0]
    
    # Choose Bottom
    best_bottom = bottom_ranks[0]
    
    # Choose Outer (only if Warmth requirement is 3 or higher)
    if required_scores['warmth'] >= 3 and outer_ranks:
        best_outer = outer_ranks[0]
    else:
        best_outer = None
        
    # --- Final Output ---
    
    outfit = {
        "Outfit Type": "Layered",
        "Weather": weather_data,
        "Required Scores": required_scores,
        "Outerwear": best_outer,
        "Top": best_top,
        "Bottom": best_bottom,
    }
    
    return outfit

In [24]:
import json
from datetime import datetime

# --- Configuration ---
TARGET_CITY = "Seoul"
OUTPUT_FILE_NAME = "recommended_outfit.txt"

# --- RUN RECOMMENDATION ---
recommended_outfit = recommend_outfit(df_wardrobe, TARGET_CITY)

# Open the file for writing ('w')
with open(OUTPUT_FILE_NAME, 'w') as f:
    
    # --- Write Header ---
    f.write(f"--- OUTFIT RECOMMENDATION for {TARGET_CITY} ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')}) ---\n\n")

    # --- Write Detailed JSON Result ---
    f.write("--- DETAILED JSON OUTPUT ---\n")
    if isinstance(recommended_outfit, dict):
        # Write the JSON string to the file
        f.write(json.dumps(recommended_outfit, indent=4))
        f.write("\n\n")

        # --- Write Readable Summary ---
        f.write("--- READABLE SUMMARY ---\n")
        
        if recommended_outfit['Outfit Type'] == 'Layered':
            f.write(f"Outfit Type: Layered\n")
            
            # Write Outerwear
            outerwear_note = "None recommended (weather is warm)."
            outerwear_warmth = "N/A"
            if recommended_outfit["Outerwear"]:
                outerwear_note = recommended_outfit['Outerwear']['notes']
                outerwear_warmth = recommended_outfit['Outerwear']['warmth_score']
            f.write(f"Outerwear: {outerwear_note} (Warmth: {outerwear_warmth})\n")
            
            # Write Top
            top_note = recommended_outfit['Top']['notes']
            top_warmth = recommended_outfit['Top']['warmth_score']
            f.write(f"Top:       {top_note} (Warmth: {top_warmth})\n")
            
            # Write Bottom
            bottom_note = recommended_outfit['Bottom']['notes']
            bottom_warmth = recommended_outfit['Bottom']['warmth_score']
            f.write(f"Bottom:    {bottom_note} (Warmth: {bottom_warmth})\n")
        
        elif recommended_outfit['Outfit Type'] == 'Dress':
            # Write Dress
            dress_note = recommended_outfit['Dress']['notes']
            dress_warmth = recommended_outfit['Dress']['warmth_score']
            f.write(f"Dress:     {dress_note} (Warmth: {dress_warmth})\n")
            
    else:
        # Handle the error case (if recommend_outfit returned a string error message)
        f.write(f"Recommendation Failed: {recommended_outfit}\n")
        f.write("Please verify your API key and city name.")

print(f"✅ Recommendation saved successfully to {OUTPUT_FILE_NAME}")

Weather in Seoul: 1.8°C, clear sky

Required Scores: {'warmth': 5, 'impermeability': 1, 'layering': 4}

Top Ranks: [{'image_link': 'simulated_wardrobes/Female_Wardrobe/Tops_09_Polo_50672a01-02ba-4583-8ce1-dda8b1884820.jpg', 'category': 'button-up shirt', 'outer_inner': 'inner', 'shape': {'sleeve': 'short-sleeve', 'neckline': 'collar', 'fit': 'regular'}, 'material': 'knit', 'color': 'dark blue with light blue stripes', 'pattern': 'striped', 'warmth_score': 4, 'layering_score': 4, 'impermeability_score': 1, 'comfort_score': 2, 'notes': 'A dark blue knit polo shirt with thin light blue horizontal stripes and a pointed collar.', 'warmth_fit': 9, 'impermeability_fit': 10, 'layering_fit': 4, 'total_score': 8.3}, {'image_link': 'simulated_wardrobes/Female_Wardrobe/Tops_05_Kaos_4ab46ac8-9031-4a33-8930-c6fdaafb420b.jpg', 'category': 't-shirt', 'outer_inner': 'inner', 'shape': {'sleeve': 'short-sleeve', 'neckline': 'crew-neck', 'fit': 'regular'}, 'material': 'synthetic (nylon/polyester)', 'color